# Alignment with regular matrix (not masked)

## What next?

In cell 10 we check each position, but if the new vector adds no new information
we don't have to check any positions, and if it adds only some new information, we
have to check only the positions where that happens. So:

Create bitarrays for each vector as we create them and compare the bitarrays first. That
will tell us when we have to do nothing and when we have to check only some positions.
The cost is that we have to maintain the bitarrays; the payoff is that checking them is
much faster than walking over the vectors position by position. The payoff is especially
great when the new vector adds no new information.

We can count the number of times this happens first to see how much the payoff would be.

Another potential optimization:
Maximal (longest) frequent (often occurring) sequences would group blocks together. MFS
are discontinuous, finding them takes time, but they would let us reduce the number of
blocks, and therefore the number of merges. Question: does discontinuity complicate 
transposition detection.

Profiling transposition detection with `%lprn` errors out, and it shouldn’t. Why?

## Initialize

In [1]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict, deque
from typing import Set, List
from dataclasses import dataclass
from bitarray import bitarray
import networkx as nx
import re
import queue

import numpy as np
import numba as nb
import numpy.ma as ma

import graphviz
from IPython.display import SVG

from numba import jit

import pprint
pp = pprint.PrettyPrinter(indent=2)
debug = False

%load_ext line_profiler

In [2]:
# Identity set
# https://stackoverflow.com/questions/16994307/identityset-in-python (first solution)
from collections.abc import MutableSet

class IdentitySet(MutableSet):
    key = id  # should return a hashable object

    def __init__(self, iterable=()):
        self.map = {} # id -> object
        self |= iterable  # add elements from iterable to the set (union)

    def __len__(self):  # Sized
        return len(self.map)

    def __iter__(self):  # Iterable
        return iter(self.map.values())

    def __contains__(self, x):  # Container
        return self.key(x) in self.map

    def add(self, value):  # MutableSet
        """Add an element."""
        self.map[self.key(value)] = value

    def discard(self, value):  # MutableSet
        """Remove an element.  Do not raise an exception if absent."""
        self.map.pop(self.key(value), None)

    def __repr__(self):
        if not self:
            return '%s()' % (self.__class__.__name__,)
        return '%s(%r)' % (self.__class__.__name__, list(self))

## Load data

In [3]:
%run 20220123_one-based_create-blocks-and-dependencies.ipynb # data will be in prioritized_blocks
# print(dependencies)

## Create witness_node (dataclass)

In [4]:
@dataclass(unsafe_hash=True)
class WitnessNode:
    """TODO: Write a docstring some day"""
    token_string: str
    witness: str
    witness_offset: int

    def __repr__(self):
        return self.token_string

# Subclass of WitnessNodeEnd so that our constructed end node will be able to identify itself
class WitnessNodeEnd(WitnessNode):
    """TODO: Write a docstring some day"""

In [5]:
# Store all witness nodes in dictionary of lists
witness_node_lists = defaultdict(list) # one per witness (keys will be sigla)
# create, for each witness:
#   nodes for real tokens
#   END node
# A start node is not created. That is on purpose.
for index, witness_siglum in enumerate(witness_sigla):
    # witness_sigla is a global set when the input data is read
    # items in lists have item.token_string, item.witness, and one-based item.witness_offset properties
    for witness_token_offset, witness_token in enumerate(witnesses[index]): # list of tokens in a single witness
        witness_node_lists[witness_siglum].append(WitnessNode(witness_token, witness_siglum, witness_token_offset + 1))
    witness_node_lists[witness_siglum].append(WitnessNodeEnd('END', witness_siglum, len(token_array)))

## Create candidate vectors from block

In [6]:
def create_candidate_vectors(block):
    """Returns list of data for block, one numpy array per hyperedge"""
    candidate_vectors = [] # list of individual vectors, not a matrix
    for token_offset_in_block in range(block.token_count):
        # first add the token_offset_in_block to the block start positions
        offset_start_positions = [value + token_offset_in_block for value in block.all_start_positions]
        tokens_to_place = [ # list of all token positions, not just first in each witness
            (
                token_membership_array[token_array_index],
                token_witness_offset_array[token_array_index]
            )
            for token_array_index in offset_start_positions
        ]
        new_vector = np.array([0] * len(witness_sigla)) # initialize numpy array to all zeroes
        for witness_number, witness_offset in tokens_to_place:
            new_vector[witness_number] = witness_offset
        candidate_vectors.append(new_vector)

    return candidate_vectors

## Place vectors if allowed

In [7]:
@nb.njit(fastmath=True)
def check_single_vector_for_transposition(old_vector, potential_vector):
    """Called by check_whether_okay_to_place()

    Separated for numba compilation
    """
    subtraction_results = np.array([0] * old_vector.size)
    for column_offset, (old_value, new_value) in enumerate(zip(old_vector, potential_vector)): # compare each position
        subtraction_results[column_offset] = 0 if old_value * new_value == 0 else old_value - new_value
    signs = np.sign(subtraction_results) # values are all -1, 0, or 1
    unique_row_values = set(signs) # all unique sign values
    if 0 in unique_row_values:
        unique_row_values.remove(0) # all unique values after removing 0 (0, 1, or 2 values)
    return unique_row_values

In [8]:
def check_whether_okay_to_place(current_vectors, potential_vector) -> bool:
    """Return True iff we can add row without creating transpositions

    current_vectors: identity set of all vectors currently placed
    potential_vector: we check whether this can be added

    If subtracting a potential vector from any existing vector would return
    values that diverge in sign, the potential would cross the existing one

    Instead of broadcast subtraction, we use plain ol' Python and trust numba
    to optimate:
        We compile check_single_value_for_transposition(), since we can't
        optimate the extendable list here

    If either value in a column is 0 (placeholder, not a real position), return 0
    Otherwise perform subtraction

    If it's okay to place we **can** merge, but we need to call merge_vector()
        to see whether we **should** merge"""

    ###
    # Can we optimize? Check signage of each row before subtraction for the next
    # This lets us bail out as soon as a row has mixed signs, since that means
    #   transposition, and we can avoid performing complete subtraction
    # But: transpositions don't happen often, so there may little to gain from
    #   the extra effort
    ###
    #
    for vector_to_check in current_vectors:
        unique_row_values = check_single_vector_for_transposition(vector_to_check, potential_vector)
        if len(unique_row_values) == 2: # if any row has pos and neg, there's a transpostion, so bail out
            return False
    return True # if we get this far, no transposition in any row

In [9]:
@nb.njit
def merge_vectors(old_data, new_vector):
    """Add a docstring some day

    Parameters:
        old_data: vector that contains consolidated data from mergeable rows
        new_vector: vector
        Values in each position are either 0 or witness offset
    """
    return np.maximum(old_data, new_vector)

In [10]:
def merge_vectors_in_place(old_data, new_vector, witness_lists):
    for pos, value in enumerate(new_vector):
        if old_data[pos] == 0 and value != 0:
            old_data[pos] = value
            witness_lists[pos][value] = old_data

In [11]:
def add_new_vector(witness_lists, consolidated_identity_set, new_vector):
    """Add row to matrix and update pointer to next empty row

    To add a vector:
        Add to all non-zero witness lists and to consolidated list
        Not a merge, so we add (no replace)

    Parameters:
        witness_lists: tuple of lists (write to specific offset of each)
            order of witness lists matches order of columns in vector
            first (zeroeth) position is always None because witness offset positions are 1-based
        consolidated_identity_set: IdentitySet of all placed vectors
        new_vector : vector to add

    Void because lists contain pointers, so change in place
    """
    consolidated_identity_set.add(new_vector)
    for witness_number, witness_token_offset in enumerate(new_vector):
        if witness_token_offset != 0:
            witness_lists[witness_number][witness_token_offset] = new_vector

In [12]:
def add_or_merge_new_vector_into_matrix(witness_lists, consolidated_identity_set, new_vector):
    """Do stuff

    Parameters:
        witness_lists: tuple of lists (write to specific offset of each)
            order of witness lists matches order of columns in vector
            first (zeroeth) position is always None because witness offset positions are 1-based
        consolidated_identity_set: IdentitySet of all placed vectors
        new_vector : vector to add

    Void because lists contain pointers, so change in place
    """

    # Query witness-specific lists to collect merge candidates
    merge_candidates = IdentitySet()
    for witness_number, witness_token_offset in enumerate(new_vector):
        # FIXME: we do the same lookup twice; use a walrus
        if witness_token_offset != 0 and witness_lists[witness_number][witness_token_offset] is not None:
            merge_candidates.add(witness_lists[witness_number][witness_token_offset])

    if len(merge_candidates) == 0: # no merge candidate, so add candidate as new vector
        add_new_vector(witness_lists, consolidated_identity_set, new_vector)

    elif len(merge_candidates) == 1: # one merge candidate, so (relatively) simple merge
        merge_vectors_in_place(merge_candidates.pop(), new_vector, witness_lists)

    else: # multiple merge candidates
        # perform merge in new array
        # remove old candidates from consolidated_identity_set
        result = np.array([0] * len(new_vector)) # will hold merge result as vector
        for merge_candidate in merge_candidates: # identity set of vectors
            for pos, value in enumerate(merge_candidate): # check each position
                if result[pos] == 0 and value != 0: # update only when necessary
                    result[pos] = value
            consolidated_identity_set.discard(merge_candidate)
        # update non-zero values witness-specific lists
        for witness_number, witness_token_offset in enumerate(result):
            if witness_token_offset != 0:
                witness_lists[witness_number][witness_token_offset] = result
        # add merge result to consolidated_identity_set
        consoidated_identity_set.add(result)

## Process blocks

In [13]:
# Create witness-specific lists and consolidated_identity_set here
#
def process_blocks(selected_blocks):
    """ Add all data to witness-specific lists of vectors and to consolidated identity set

    Return only consolidated identity set
    """
    #selected_blocks is a list of tuples, each of which is a Block and an integer, representing its dependency position
    dependency_bitarray = bitarray(len(selected_blocks)) # track dependencies
    dependency_bitarray.setall(False) # true = no need to place; initially all false
    # ##
    # initialze witness lists and consolidated identity set
    # ##
    witness_lists = [] # will convert to tuple
    for witness_position in range(len(witnesses)):
        witness_lists.append([None] * (len(witnesses[witness_position]) + 1))
    witness_lists = tuple(witness_lists) # stable, so now convert to tuple
    consolidated_identity_set = IdentitySet()
    # ##
    # process blocks
    # ##
    for current_block, dependency_pointer in selected_blocks:
        if dependency_bitarray[dependency_pointer]: # block (or superset) already placed, so skip
            continue
        else:
            dependency_bitarray[dependency_pointer] = True # no need to revisit this block
            # Create vectors for entire block
            new_vectors = create_candidate_vectors(current_block)
            # Check only first and last, merge is both are okay
            # NB: Checking only first and last to test entire block will break with discontinuous blocks
            #   (should we ever switch to working with discontinuous blocks)
            no_transposition = \
                check_whether_okay_to_place(consolidated_identity_set, new_vectors[0]) and \
                check_whether_okay_to_place(consolidated_identity_set, new_vectors[-1])
            if no_transposition: # all-or-nothing
                for potential_vector in new_vectors:
                    input_tuple = add_or_merge_new_vector_into_matrix(witness_lists, consolidated_identity_set, potential_vector)
                while dependency_pointer in dependencies: # TODO: alternative to updating variable ("not very beautiful")
                    dependency_bitarray[dependencies[dependency_pointer]] = True
                    dependency_pointer = dependencies[dependency_pointer]
            else:
                pass
    return consolidated_identity_set

## Main

In [14]:
# create all vectors (gingerly)
# We excluded blocks with repetition (temporarily?) before importing into this notebook

%lprun -f add_or_merge_new_vector_into_matrix alignment_vectors = process_blocks(prioritized_blocks)
# alignment_vectors = process_blocks(prioritized_blocks) # identity set of all placed alignment vectors

Timer unit: 1e-06 s

Total time: 5.21777 s

Could not find file /tmp/ipykernel_3115/3616626631.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           
     2                                           
     3                                           
     4                                           
     5                                           
     6                                           
     7                                           
     8                                           
     9                                           
    10                                           
    11                                           
    12                                           
    13                                           
    14                                        

## Create variant graph from numpy masked array

In [15]:
# Hack to adapt code written originally for numpy matrix to work with identity set instead
alignment_matrix = alignment_vectors

In [16]:
class VG_node():
    """Variant graph node"""
    def __init__(self, token_string, data): # dictionary of siglum:witness_offset
        self.token_string = token_string
        self.__dict__.update(**data)
        self._sigla = [key for key in data.keys()]
    def __repr__(self):
        return self.token_string + "~" + "|".join([":".join([str(key), str(getattr(self, key))]) for key in self.sigla()])
    def __setitem__(self, key, value):
        self._sigla.append(key)
        self.__dict__[key] = value
    def __getitem__(self, key):
        return self.__dict__[key]
    def __contains__(self, key):
        return key in self.__dict__
    def sigla(self):
        return self._sigla
    def values(self):
        return [self[key] for key in self._sigla]
    def items(self):
        return [(key, self.__dict__[key]) for key in self._sigla]

In [17]:
# Create networkx variant graph from our (not networkx) alignment graph
# Create variant graph edges for start and end nodes (only)
# We've treated VG as a global and we refer to it in the earlier functions; should we pass it instead?
def augment_or_add_edge_without_conversion(siglum, source_VG_node, target_VG_node):
    if VG.has_edge(source_VG_node, target_VG_node):
        VG[source_VG_node][target_VG_node]["siglum"].append(siglum)
    else:
        VG.add_edge(
            source_VG_node,
            target_VG_node,
            siglum = [siglum]
        )

In [18]:
# Use alignment graph node to look up variant graph node
def alignment_node_to_VG_node(alignment_node: WitnessNode) -> VG_node:
    global witness_offset_to_VG_node
    return witness_offset_to_VG_node[alignment_node.witness][alignment_node.witness_offset]

In [19]:
# Create variant graph data edges (except start and end nodes)
def augment_or_add_edge(siglum, source, target):
    source_VG_node = alignment_node_to_VG_node(source)
    target_VG_node = alignment_node_to_VG_node(target)
    if VG.has_edge(source_VG_node, target_VG_node):
        VG[source_VG_node][target_VG_node]["siglum"].append(siglum)
    else:
        VG.add_edge(
            source_VG_node,
            target_VG_node,
            siglum = [siglum]
                )

In [20]:
print(witness_node_lists['w0'][:5])
print(witness_node_lists['w0'][-5:])

[WHEN, we, look, to, the]
[by, danger,, seems, probable., END]


In [21]:
def create_variant_graph_from_vector_space(alignment_matrix):
    # create variant graph and add start and end nodes

    start_offsets = {} # make sure that START points to first tokens of each witness
    for siglum in witness_sigla:
        start_offsets[siglum] = -1;
    start_node = VG_node("START", start_offsets)
    end_node = VG_node("END", {})
    global VG # does this have to be global; we return it at the end, but we also use it in functions above
    VG = nx.DiGraph(start = start_node, end = end_node) # create start and end properties to find terminal nodes
    VG.add_node(start_node)
    VG.add_node(end_node)

    # keep track of which witness nodes (and therefore also vectors)
    # have been added to VG
    # does this duplicate information available from the following structure, since
    #   if we assign a value below to replace the None, this doesn't seem to add anything
    #   Can we get rid of this and use witness_offset_to_VG_node for this purpose instead?
    # NB: Includes END (not START), which we will later ignore (we think …)
    from bitarray import bitarray
    global VG_tracking # keep this, but should we return it instead of making it global?
    VG_tracking = {}
    for siglum, witness_node_list in witness_node_lists.items():
        VG_tracking[siglum] = bitarray(len(witness_node_list))
        VG_tracking[siglum].setall(0)

    # map from witness token node to variant graph node (needed to construct edges)
    # values will be added as we create variant graph nodes
    # NB: Includes END (not START), which we will later ignore (we think …)
    global witness_offset_to_VG_node # does this have to be global?
    witness_offset_to_VG_node = {}
    for siglum in witness_node_lists.keys():
        witness_offset_to_VG_node[siglum] = [None] * (len(witness_node_lists[siglum]))

    # Replaces old code in following cell to build variant graph,
    #   this time using vector space as main data source

    # 1. Convert vectors to variant graph nodes

    for row in alignment_matrix: # row is a numpy array, with 0 for "masked" values
        if debug:
            print(' '.join(('Processing', str(row))))
        data = {} # create dictionary of siglum:value for new VG node (variable name: data) and update globals
        for index, value in enumerate(row):
            if value != 0:
                siglum = sigla[index]
                value = value - 1 # alignment vector offsets are one greater than offsets in witness_node_lists
                data[siglum] = value # add to data for eventual new VG node
                VG_tracking[siglum][value] = 1 # update global
        # get token string for node (variable name: token_string)
        siglum, offset = next(iter(data.items())) # get value without depleting original dictionary
        token_string = witness_node_lists[siglum][offset].token_string
        # create and add new VG_node
        new_VG_node = VG_node(token_string, data)
        VG.add_node(new_VG_node)
        for siglum, offset in data.items(): # Eek! Another for loop! How embarrassing!
            witness_offset_to_VG_node[siglum][offset] = new_VG_node # update other global;

    # 2. Traverse variant graph (in arbitrary order), each of which contains witness nodes.
    #    Draw outgoing edges, which point to nodes with next tokens in each witness present
    #      on the variant graph node.
    #    Creating new variant graph nodes for witness tokens not in a alignment vector.
    #    Use queue because we're iterating over a dynamic structure (inventory of variant graph nodes).

    VG_node_queue = queue.Queue()
    #
    # Temporarily skipping START and END, which are the first two nodes
    #
    for node in VG.nodes(): # add all initial VG nodes to queue except START and END
        VG_node_queue.put(node)
        if debug:
            print(f"Adding node to VG_node_queue; length is {VG_node_queue.qsize()=}")
#     ignore_start = VG_node_queue.get() # temporarily ignore START node when creating edges
#     ignore_end = VG_node_queue.get() # temporarily ignore END node when creating edges
#     if debug:
#         print(f"Removed START and END; length is {VG_node_queue.qsize()=}")
    while not VG_node_queue.empty():
        if debug:
            print(f"Processing source node from VG_node_queue; length is {VG_node_queue.qsize()=}")
        source_node = VG_node_queue.get()
        targets = set()
        edge_labels = defaultdict(list) # key is target VG node, value is list of sigla
        source_sigla = source_node._sigla # all sigla on source node
        for siglum in source_sigla:
            # target may or may not already exist as node in VG
            source_offset = source_node[siglum]
            target_offset = source_offset + 1
            target = witness_offset_to_VG_node[siglum][target_offset]
            if target: # does the target already exist?:
                targets.add(target)
                edge_labels[target].append(siglum)
            else: # 1) create target, 2) add to targets, 3) add to queue of nodes, and 4 + 5) update globals
                # TODO: Do we need to update globals? We shouldn't need to return to nodes we add here.
                # NB: If real target object is of type WitnessNodeEnd, don't add it to the queue or create an edge
                # print(f"{type(witness_node_lists[siglum][target_offset])=}")
                witness_node_target = witness_node_lists[siglum][target_offset]
                if type(witness_node_target) != WitnessNodeEnd:
                    target_token_string = witness_node_target.token_string
                    new_VG_node = VG_node(target_token_string, {siglum: target_offset}) # 1 create target
                    targets.add(new_VG_node) # 2 add to targets
                    edge_labels[new_VG_node].append(siglum)
                    VG_node_queue.put(new_VG_node) # 3 add to queue of nodes
                    VG_tracking[siglum][target_offset] = 1 # 4 update first global
                    witness_offset_to_VG_node[siglum][target_offset] = new_VG_node # 5. update second global
        for target in targets:
            VG.add_edge(source_node, target, label=",".join(edge_labels[target]))
    #
    # Add edges for end VG node
    #
    all_end_data_nodes = defaultdict(list)
    for siglum in witness_sigla:
        # Figure out why -2 works and clean up as needed
        key = witness_offset_to_VG_node[siglum][-2]
        all_end_data_nodes[key].append(siglum)
    for key,value in all_end_data_nodes.items():
        VG.add_edge(key, end_node, label=",".join(value))

    # we're done! return the result
    return VG

In [22]:
# create output
VG = create_variant_graph_from_vector_space(alignment_matrix)

In [23]:
# alignment_matrix.shape

## Visualize variant graph

In [24]:
# # node id values must be strings for graphviz
# a = graphviz.Digraph(format="svg", name="variant_graph_unjoined")
# a.attr(rankdir = "LR")
# a.attr(rank = 'same')
# a.attr(compound='true')

# # plot nodes, building {node: id} for lookup
# node_to_id = {}
# for index, node in enumerate(VG.nodes()):
#     node_id = str(index)
#     node_to_id[node] = node_id
#     if node is None:
#         node_text = 'NONE'
#     else:
#         node_text = node.token_string + " (" + node_id + ")"
#     a.node(node_id, label=node_text)

# # plot edges
# for edge in VG.edges(data=True):
#     # edge is a three-item tuple: source, target, dictionary of properties
#     a.edge(node_to_id[edge[0]], node_to_id[edge[1]], label=edge[2]["label"])

# print('aligning', how_many_paragraphs, 'paragraphs') # confirm
# SVG(a.view())

In [25]:
#  This function joins the variant graph in place.
#  This function is a straight port of the Java version of CollateX.

def join(graph):
    processed = set()
    end = graph.graph["end"]
    queue = deque()
    for (_, neighbor) in graph.out_edges(graph.graph["start"]):
        queue.appendleft(neighbor)
    while queue:
        vertex = queue.popleft()
        out_edges = graph.out_edges(vertex)
        if len(out_edges) == 1:
            (_, join_candidate) = next(iter(out_edges))
            can_join = join_candidate != end and len(graph.in_edges(join_candidate)) == 1
            if can_join:
                join_vertex_and_join_candidate(graph, join_candidate, vertex)
                # we have merged join_candidate (tokens originally to the right) into vertex (tokens originally to the left)
                # (for now, join_candidate node and all of its edges are still there, and we need to remove edges before
                #   we can remove node, which we do in a for loop)
                #
                # RESUME HERE: both of the following branches are wrong, the first cryptically (it shows sigla, but not
                #   all sigla) and the second conspicuously
                #
                for (_, neighbor, data) in list(graph.out_edges(join_candidate, data=True)):
                    graph.remove_edge(join_candidate, neighbor)
                    if graph.has_edge(vertex, neighbor):
                        continue # TODO: this looks wrong
#                         graph[source][target]["siglum"].append(data["siglum"])
                    else:
#                         graph.add_edge(vertex, neighbor, siglum=data["siglum"])
                         graph.add_edge(vertex, neighbor,label=data["label"])
                graph.remove_edge(vertex, join_candidate)
                graph.remove_node(join_candidate)
                queue.appendleft(vertex)
                continue
        processed.add(vertex)
        for (_, neighbor) in out_edges:
            # FIXME: Why do we run out of memory in some cases here, if this is not checked?
            if neighbor not in processed:
                queue.appendleft(neighbor)


def join_vertex_and_join_candidate(graph, join_candidate, vertex):
    # Note: since there is no normalized/non normalized content in the graph
    # a space character is added here for non punctuation tokens

    if re.match(r'^\W', join_candidate.token_string):
        vertex.token_string += join_candidate.token_string
    else:
        vertex.token_string += (" " + join_candidate.token_string)
    # join_candidate must have exactly one token (inside a list); left item may have more
#     for siglum, token in join_candidate.tokens.items():
#         vertex.add_token(siglum, token[0])

In [26]:
# len(nx.algorithms.cycles.find_cycle(VG))

In [27]:
# nx.algorithms.cycles.find_cycle(VG)

In [28]:
# We need this to create a joined variant graph
# Commented out temporarily
#
join(VG)

# node id values must be strings for graphviz
a = graphviz.Digraph(format="svg", name="variant_graph_joined")
a.attr(rankdir = "LR")
a.attr(rank = 'same')
a.attr(compound='true')

# plot nodes, building {node: id} for lookup
node_to_id = {}
for index, node in enumerate(VG.nodes()):
    node_id = str(index)
    node_to_id[node] = node_id
    if node is None:
        node_text = 'None'
    else:
        node_text = node.token_string + " (" + node_id + ")"
    a.node(node_id, label=node_text)

# plot edges
for edge in VG.edges(data=True):
    # edge is a three-item tuple: source, target, dictionary of properties
#     label = "(all)" if len(edge[2]["siglum"]) == len(witness_sigla) else ",".join(sorted(edge[2]["siglum"]))
    label = edge[2]["label"]
    a.edge(node_to_id[edge[0]], node_to_id[edge[1]], label=label)

print('aligning', how_many_paragraphs, 'paragraphs') # confirm
SVG(a.view())

aligning 3 paragraphs


In [29]:
# Find first non-masked value in each column to avoid traversing entire column
# In our matrix, all non-masked column values for rows that are merge candidates
#   have the same value. Previously we found max(), which had to look at them all
#   and compare, but the first value will necessarily also be the max().

# current_matrix = ma.MaskedArray(
#     data =[[-1, 2, 3], [4, -1,6], [7,8,-1]],
#     mask=[[True, False, False],[False, True, False],[False, False, True]],
#     fill_value=-1
# )
# columns = current_matrix.T # swap rows and columns and process (new) rows
# [print(column) for column in columns]
# first_non_masked = [column[column.mask == False][0] for column in columns]
# first_non_masked

In [30]:
tmp = iter(VG.nodes)
for i in range (5):
    print(next(tmp))
    print()

START~w0:-1|w1:-1|w2:-1|w3:-1

END~

when the organisation has once begun to vary, it generally continues to vary for many generations. No case is on record of a variable being ceasing to be variable under cultivation. Our oldest cultivated plants, such as wheat, still often yield new varieties: our oldest domesticated animals are still capable of rapid improvement or modification. It has been disputed at what period of life the causes of variability, whatever they may be, generally act; whether during the early or late period of development of the embryo, or at the instant of conception. Geoffroy St. Hilaire's experiments show that unnatural treatment of the embryo causes monstrosities; and monstrosities cannot be separated by any clear line of distinction from mere variations. But I am strongly inclined to suspect that the most frequent cause of variability may be attributed to the male and female reproductive elements having been affected prior to the act of conception. Several reas